# **Brief 6 - data vizualisation**

## **0) Nettoyage du jeux de données**  
Un fichier reprenant le frame nettoyer a été créer. Le fichier original a été sauvegardé. Le notebook utilise le nouveau fichier.

In [1]:
import numpy as np # algèbre linéaire
import pandas as pd # procès de données, CSV file I/O (e.g. pd.read_csv)

<span style="color: #FF0000">  
    
**Problème avec de split de wordCloud , remplacer les espaces par des espaces insécables ?** 
    </span>

In [2]:
import pandas as pd

# Chargement des données qui seront utilisées.
timesData = pd.read_csv("data/timesData_Original.csv")
# information about timesData
timesData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2603 non-null   object 
 1   university_name         2603 non-null   object 
 2   country                 2603 non-null   object 
 3   teaching                2603 non-null   float64
 4   international           2603 non-null   object 
 5   research                2603 non-null   float64
 6   citations               2603 non-null   float64
 7   income                  2603 non-null   object 
 8   total_score             2603 non-null   object 
 9   num_students            2544 non-null   object 
 10  student_staff_ratio     2544 non-null   float64
 11  international_students  2536 non-null   object 
 12  female_male_ratio       2370 non-null   object 
 13  year                    2603 non-null   int64  
dtypes: float64(4), int64(1), object(9)
memor

In [15]:
df_clean=timesData


#-------------------------------------------------------------------------------
#
# Nettoyage des données: remplacer les pays mal orthographié (2 de trouvé)
# remplacer les '-' par des Nan
#
#-------------------------------------------------------------------------------

# pays:
df_clean["country"]= df_clean["country"].replace({'Unisted States of America':'United States of America',
                                                 'Unted Kingdom':'United Kingdom'})
#print(df_clean["country"].unique())

# income:
df_clean["income"]= df_clean["income"].replace({'-':None})
#print(df_clean["income"].sort_values())

# total_score
df_clean["total_score"]= df_clean["total_score"].replace({'-':None})
#print(df_clean["total_score"].tail(30))

# international
df_clean["international"]= df_clean["international"].replace({'-':None})
#print(df_clean["international"].head(30))

print("Les champs research et citations sont OK")

# Le champ num_students
print("Le champs num_students pose 2 problème: Effectif non renseigné et la ',' en sépparation des miliers \
avec un risque d'interprétation ou 99,345 serait inférieur à 100 !!!")

df_clean.num_students = [str(each).replace(',', '') for each in df_clean.num_students]
df_clean.num_students  = [str(each).replace('nan', '') for each in df_clean.num_students]
#num_students  = [int(each) for each in num_students ]

#print(df_clean.num_students)

df_clean.to_csv("data/timesData.csv")

print("Le problème de la colonne 'world_rank' n'a pas été résolu. Celle-ci contient soit un nombre soit une\
      plage de données. On pourrait remplacer cette plage par une moyenne. Ou supprimer mettre à 'none' ce \
      champs lorsqu'il donnée sous forme de plage.")
print("On note que les données non numérique sont automatiquement convertie en Nan ")
print("La colonne proportion F/H pose égalemetn problème. Il faudrait la remplacer par un float. pour cela \
utilisation de regex pour extraire xx:yy et faire xx/yy")


Les champs research et citations sont OK
Le champs num_students pose 2 problème: Effectif non renseigné et la ',' en sépparation des miliers avec un risque d'interprétation ou 99,345 serait inférieur à 100 !!!
0       20152
1        2243
2       11074
3       15596
4        7929
        ...  
2598    21958
2599    31268
2600     4122
2601    10117
2602     8663
Name: num_students, Length: 2603, dtype: object
Le problème de la colonne 'world_rank' n'a pas été résolu. Celle-ci contient soit un nombre soit une      plage de données. On pourrait remplacer cette plage par une moyenne. Ou supprimer mettre à 'none' ce       champs lorsqu'il donnée sous forme de plage.
On note que les données non numérique sont automatiquement convertie en Nan 


## **Début du brief**

In [ ]:
# plotly
# import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# librairie word cloud
from wordcloud import WordCloud

# librairie matplotlib
import matplotlib.pyplot as plt

"""
# Permet d'afficher les données disponibles dans le répertoire data
import os
print(os.listdir(".")) 
"""


Contexte du projet
Classement des universités

Le classement des universités est une pratique difficile, politique et controversée. Il existe des centaines de systèmes de classement universitaires nationaux et internationaux différents, dont beaucoup sont en désaccord les uns avec les autres.

Le Times Higher Education World University Ranking est largement considéré comme l'une des mesures universitaires les plus influentes et les plus largement observées. Fondée au Royaume-Uni en 2010, elle a été critiquée pour sa commercialisation et pour avoir "affaibli" les établissements non anglophones.


ACP

Pour vous aider dans votre analyse du jeux de données, vous réaliserez une Analyse en Composantes Principales.

Cette analyse permettra de répondre à certaines questions du type : quelles ressemblances peut-il y avoir d'une université à une autre. Quelles ressemblances existent-il entre différents critères d'évaluation des universités ? Vous pourrez ainsi définir quand est-ce que 2 universités se ressemblent et quand est-ce qu'elles se ressemblent du point de vue de l'ensemble des colonnes (c'est-à-dire des critères d'évaluation du Times Higher Education World University Ranking).

Est-il possible de faire un bilan des ressemblances ? ( Vous chercherez ici à faire une typologie, une partition des universités, c'est-à-dire à construire des groupes d'universités homogènes du point de vue de l'ensemble des variables. A l'intérieur d'un groupe, les individus se ressemblent et d'un groupe à l'autre ils sont différents.
Modalités pédagogiques

    Réaliser une veille sur la librairie Dash.

    Faire une analyse du jeu de données correspondant au classement des 50 meilleures universités en 2016.

    Réaliser une Analyse en Composantes Principales (vous pourrez vous appuyer sur la librairie Scikit-Learn) 

    Mettre en place un Dashbord Dash multi-pages permettant de répondre à la question initiale : De toutes les universités du monde, quelles sont les meilleures ? La première page de votre Dashbord mettra en évidence l'analyse des données des 50 meilleures universités de l'année 2016 (avant L'ACP).

1ère page

    Dans cette première page se trouvera notamment une table des données des 50 meilleurs universités de l'année 2016 avec un bouton de téléchargement permettant de télécharger un tableau .csv des données.

    Plusieurs graphiques mettant en évidences des corrélations entre certains critères (par exemple : qualité de la recherche en fonction du rand mondial du classement des universités etc..)

2eme page

Cette page permettra d'afficher les résultats issus de l'ACP. On pourra ainsi y trouver des graphiques ainsi que des paragraphes de textes mettant en évidence des variables explicatives (par exemple la valeur propre, la proportion, le cumulé, les composantes principales (CP), les scores, les distances).

    Vous mettrez en ligne votre Dashboard Dash sur le serveur Cloud Heroku

Modalités d'évaluation

Un rendu individuel est demandé. Vous pourrez travailler en groupe de 5 ou 6. Le rendu final devra être envoyé le vendredi 4 novembre à 15h30.
